# Cas test debit

In [ ]:
from trustutils import run

run.introduction("G. F.")
run.description("Calculs menes en VEF ou VDF.")
run.TRUST_parameters()

In [ ]:
from trustutils import run 
run.addCase(".","test_opt.data")
run.addCase(".","test_opt2.data")
run.addCase(".","test_opt3.data")
run.addCase(".","Champ_front_debit.data")
run.addCase(".","champ_normal_var.data")
run.printCases()
run.runCases()

### References
* moi_meme

## VEF test:  calculated flow rate

In [ ]:
from trustutils import plot
import numpy as np

columns=["temps",'debit']
Tableau=plot.Table(columns)  

data=plot.loadText("test_opt_pb_Debit.out",transpose=False)
data[-1][4] = -data[-1][4]
Tableau.addLigne([data[-1][0:5:4]],"Calcul")

data=plot.loadText("test_opt2_pb_Debit.out",transpose=False)
data[-1][4] = -data[-1][4]
Tableau.addLigne([data[-1][0:5:4]],"Calcul avec champ_input")

data=plot.loadText("test_opt3_pb_Debit.out",transpose=False)
data[-1][4] = -data[-1][4]
Tableau.addLigne([data[-1][0:5:4]],"Calcul avec champ_front_fonc_txyz")

data=plot.loadText("champ_normal_var_pb_Debit.out",transpose=False)
data[-1][4] = -data[-1][4]
Tableau.addLigne([data[-1][0:5:4]],"Calcul avec Boundary_field_inward variable")

Tableau.addLigne([[10,5.]],"voulu") 
display(Tableau)

### Visualization of the velocity field on boundary IN 

The velocity should be uniform even if the surfaces are different.

In [ ]:
from trustutils import visit 

a = visit.Show("./test_opt.lata","Vector","VITESSE_ELEM_IN",mesh="IN")
a.normal3D([-0.0543768,-0.677396,0.733606])
a.up3D([-0.407612, 0.685749,0.602993])
a.plot()

In [ ]:
from trustutils import plot
import numpy as np

Graphique=plot.Graph()
data=plot.loadText("test_opt3_pb_Debit.out")
Graphique.add(data[0],-data[4],marker="-|",color="r",label="Calculation with imposed flow rate")
data=plot.loadText("champ_normal_var_pb_Debit.out")
Graphique.add(data[0],-data[4],marker="-x",color="g",label="Calculation with imposed velocity")
x=np.linspace(0,10,10)
y = 1.+4.*x/10.
Graphique.add(x,y,marker="-",color="b",label="(1+4*x/10)")
Graphique.label("t","debit")


### VDF test:  calculated flow rate

In [ ]:
from trustutils import plot

columns=["temps",'debit']
Tableau=plot.Table(columns)  
data=plot.loadText("Champ_front_debit_pb_Debit.out",transpose=False)
data[2][1] = -data[2][1]
Tableau.addLigne([data[2][0:2]],"Calculation with imposed flow rate")
data=plot.loadText("testCL_pb_Debit.out",transpose=False) 
data[2][1] = -data[2][1]
Tableau.addLigne([data[2][0:2]],"Calculation with imposed velocity") 
Tableau.addLigne([[1.2,5.]],"voulu") 
display(Tableau)

## Data Files

### test_opt

In [ ]:
run.dumpDataset("./test_opt.data") 